In [1]:
from fewshot.data_provider.dataset import Dataset
from fewshot.backbones import ConvNet

from fewshot.algorithms.fewshot_models import BaselineFewShotModel
from tensorflow.keras import callbacks

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import scipy.stats as st
import cv2

import tensorflow as tf
import os
import pandas as pd
from fewshot.tester import Tester

from fewshot.algorithms.backbone_train import (
    simple_one_layer_cross_entropy_train,
    simple_cosine_layer_cross_entropy_train
)

from fewshot.algorithms.fewshot_test import baseline_fewshot_test, bootstrap

from IPython.display import clear_output

### Init

Set up all input data and params

In [ ]:
!python3 prepare_dataset.py omniglot --dataset_root '../fewshot/datasets/'

Preprocessing omniglot as in the $\href{https://openreview.net/pdf?id=HkxLXnAcFQ}{article}$

In [ ]:
def augment_omniglot(omniglot_dir):
    file = open(os.path.join(omniglot_dir, 'data.csv'))
    smth = pd.read_csv(file)
    file.close()
    appendix = []
    for i in tqdm_notebook(range(smth.shape[0])):
        string = smth.loc[i]
        filepath = string['filepath']
        class_ = string['class']
        super_class = string['super_class']
        subset = string['subset']
        filepath = os.path.join(omniglot_dir, filepath)
        img = cv2.imread(filepath)
        img90 = cv2.rotate(img, rotateCode=0)
        img180 = cv2.rotate(img, rotateCode=1)
        img270 = cv2.rotate(img, rotateCode=2)
        fp = filepath.split('/')
        name_90 = fp[-1].split('.')[0] + '_flip_90.png'
        path_90 = os.path.join('/'.join(fp[:-1]), name_90)
        name_180 = fp[-1].split('.')[0] + '_flip_180.png'
        path_180 = os.path.join('/'.join(fp[:-1]), name_180)
        name_270 = fp[-1].split('.')[0] + '_flip_270.png'
        path_270 = os.path.join('/'.join(fp[:-1]), name_270)
        cv2.imwrite(path_90, img90)
        cv2.imwrite(path_180, img180)
        cv2.imwrite(path_270, img270)
        appendix.append({
            'filepath': path_90.split(omniglot_dir)[1],
            'class': class_ + '_flip_90',
            'super_class': super_class,
            'subset': subset
        })
        appendix.append({
            'filepath': path_180.split(omniglot_dir)[1],
            'class': class_ + '_flip_180',
            'super_class': super_class,
            'subset': subset
        })
        appendix.append({
            'filepath': path_270.split(omniglot_dir)[1],
            'class': class_ + '_flip_270',
            'super_class': super_class,
            'subset': subset
        })
    appendix = pd.DataFrame(appendix, columns=['filepath', 'class', 'super_class', 'subset'])
    app = pd.merge(smth, appendix, 'outer')
    app.to_csv(os.path.join(omniglot_dir, 'data.csv'), index=False)

In [ ]:
augment_omniglot('../fewshot/datasets/omniglot/')

In [ ]:
!python3 resize_images.py "omniglot" "28, 28"

In [2]:
# common
seed = 42

img_width = 28
img_height = 28
img_depth = 3

# classes
base_num_classes = 4112
novel_num_classes = 1692
val_num_classes = 688  # not used in baseline algo

In [3]:
# fewshot training and testing
n_way = 5
k_shot = 5
fewshot_batch_size = 4
support_epochs = 100
query_size = 16

support_generator_args={
    "size": (int(img_width * 1.15), int(img_height * 1.15)),
    "center": True,  # for center cropping,
    "crop_size": (img_width, img_height),
}

query_generator_args={
    "size": (int(img_width * 1.15), int(img_height * 1.15)),
    "center": True,  # for center cropping,
    "crop_size": (img_width, img_height),
}

n_episodes = 50

In [4]:
# backbone training
backbone_training_batch_size = 16
backbone_training_epochs = 5

backbone_generator_args={
    "size": (int(img_width * 1.15), int(img_height * 1.15)),
    "center": True,  # for center cropping,
    "crop_size": (img_width, img_height),
}


In [5]:
dataset_dir = "../fewshot/datasets/omniglot/"
dataset = Dataset(dataset_dir=dataset_dir, csv_name="data_28x28.csv", image_size=(img_width, img_height))

### Backbone
Create backbone dataset (80 classes), split to train and validattion

In [6]:
backbone_dataset, val_fewshot_dataset = dataset.split_by_classes(train_size=base_num_classes,
                                                                 random_state=seed)

Split by classes with train size = 4112 (seed = 42)
Train classes: 4112
Test classes: 2380
Train data: 82240 samples
Test data:  47600 samples


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
val_dataset, fewshot_dataset = val_fewshot_dataset.split_by_classes(train_size=val_num_classes,
                                                                    random_state=seed)

Split by classes with train size = 688 (seed = 42)
Train classes: 688
Test classes: 1692
Train data: 13760 samples
Test data:  33840 samples


Create classifier

In [8]:
backbone = ConvNet(input_size=(img_width, img_height, img_depth))

Instructions for updating:
Colocations handled automatically by placer.


# Linear Layer:

In [10]:
checkpoint_dir = '../fewshot/checkpoints'
log_dir = '../fewshot/logs'

In [11]:
model_name = 'baseline'

In [12]:
backbone_optimizer = tf.keras.optimizers.Adam(lr=1e-3)

In [13]:
backbone.set_trainable(True)

In [14]:
# TODO: incapsulate inside fewshot library
backbone_callbacks = [
    tf.keras.callbacks.ModelCheckpoint("../fewshot/checkpoints/weights.{epoch:02d}-{loss:.2f}.hdf5",
                                           monitor="loss",
                                       save_best_only=False)
]

In [ ]:
simple_one_layer_cross_entropy_train(
    backbone,
    backbone_dataset.get_batch_generator(batch_size=backbone_training_batch_size,
                                         shuffle=True,
                                         generator_args=backbone_generator_args),
    epochs=backbone_training_epochs,
    optimizer=backbone_optimizer,
    model_name=model_name,
    checkpoint_dir=checkpoint_dir,
    period=5,
    tensorboard=True,
    log_dir=log_dir,
)

### Few Shot
Train fewshot model, 5-shot

In [ ]:
episode_generator = fewshot_dataset.get_fewshot_generator(n_way=n_way, 
                                                          k_shot=5, 
                                                          query_size=query_size,
                                                          support_generator_args=support_generator_args,
                                                          query_generator_args=query_generator_args)

In [ ]:
fewshot_optimizer = tf.keras.optimizers.Adam(lr=1e-3) 

In [ ]:
fewshot_model = BaselineFewShotModel(backbone, n_way, with_cosine=True)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
                                   period=10)

In [ ]:
mean_one_shot, std_one_shot, left_bound_one_shot, right_bound_one_shot = bootstrap(accuracies)

In [ ]:
print('mean:{:0.3f}, std:{:0.3f}, conf.invterval:[{:0.3f},{:0.3f}]'.format(mean_one_shot, std_one_shot, 
                                                                    left_bound_one_shot, right_bound_one_shot))

In [ ]:
fewshot_model = BaselineFewShotModel(backbone, n_way, with_cosine=False)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
                                   period=10)

In [ ]:
mean_one_shot, std_one_shot, left_bound_one_shot, right_bound_one_shot = bootstrap(accuracies)

In [ ]:
print('mean:{:0.3f}, std:{:0.3f}, conf.invterval:[{:0.3f},{:0.3f}]'.format(mean_one_shot, std_one_shot, 
                                                                    left_bound_one_shot, right_bound_one_shot))

### Few Shot
Train fewshot model, 1-shot

In [ ]:
episode_generator = fewshot_dataset.get_fewshot_generator(n_way=n_way, 
                                                          k_shot=1, 
                                                          query_size=query_size,
                                                          support_generator_args=support_generator_args,
                                                          query_generator_args=query_generator_args)

In [ ]:
fewshot_model = BaselineFewShotModel(backbone, n_way, with_cosine=True)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
#                                    log_dir=log_dir,
                                   period=10)

In [ ]:
mean_one_shot, std_one_shot, left_bound_one_shot, right_bound_one_shot = bootstrap(accuracies)

In [ ]:
print('mean:{:0.3f}, std:{:0.3f}, conf.invterval:[{:0.3f},{:0.3f}]'.format(mean_one_shot, std_one_shot, 
                                                                    left_bound_one_shot, right_bound_one_shot))

In [ ]:
fewshot_model = BaselineFewShotModel(backbone, n_way, with_cosine=False)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
#                                    log_dir=log_dir,
                                   period=10)

In [ ]:
mean_one_shot, std_one_shot, left_bound_one_shot, right_bound_one_shot = bootstrap(accuracies)

In [ ]:
print('mean:{:0.3f}, std:{:0.3f}, conf.invterval:[{:0.3f},{:0.3f}]'.format(mean_one_shot, std_one_shot, 
                                                                    left_bound_one_shot, right_bound_one_shot))

# Cosine Layer:

In [9]:
backbone = ConvNet(input_size=(img_width, img_height, img_depth))

In [10]:
checkpoint_dir = '../fewshot/checkpoints'
log_dir = '../fewshot/logs'

In [11]:
model_name = 'baseline'

In [12]:
backbone_optimizer = tf.keras.optimizers.Adam(lr=1e-3)

In [13]:
backbone.set_trainable(True)

In [14]:
backbone_callbacks = [
    tf.keras.callbacks.ModelCheckpoint("../fewshot/checkpoints/cos_weights.{epoch:02d}-{loss:.2f}.hdf5",
                                           monitor="loss",
                                       save_best_only=False)
]

In [15]:
simple_cosine_layer_cross_entropy_train(
    backbone,
    backbone_dataset.get_batch_generator(batch_size=backbone_training_batch_size,
                                         shuffle=True,
                                         generator_args=backbone_generator_args),
    epochs=backbone_training_epochs,
    optimizer=backbone_optimizer,
    model_name=model_name,
    checkpoint_dir=checkpoint_dir,
    period=5,
    tensorboard=True,
    log_dir=log_dir,
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
5140/5140 [==============================] - 508s 99ms/step - loss: 6.6147 - acc: 0.0542
Epoch 2/5
5140/5140 [==============================] - 520s 101ms/step - loss: 3.7599 - acc: 0.2838
Epoch 3/5
5140/5140 [==============================] - 557s 108ms/step - loss: 2.4400 - acc: 0.4926
Epoch 4/5
5140/5140 [==============================] - 546s 106ms/step - loss: 1.8304 - acc: 0.6045
Epoch 5/5
5140/5140 [==============================] - 538s 105ms/step - loss: 1.4910 - acc: 0.6694


### Few Shot
Train fewshot model, 5-shot, with cosine

In [16]:
episode_generator = fewshot_dataset.get_fewshot_generator(n_way=n_way, 
                                                          k_shot=5, 
                                                          query_size=query_size,
                                                          support_generator_args=support_generator_args,
                                                          query_generator_args=query_generator_args)

In [17]:
fewshot_optimizer = tf.keras.optimizers.Adam(lr=1e-3) 

In [18]:
fewshot_model = BaselineFewShotModel(backbone, n_way, with_cosine=True)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
                                   period=10)

Average acc: 97.80%: 100%|██████████| 50/50 [04:28<00:00,  7.25s/it]


In [19]:
mean_one_shot, std_one_shot, left_bound_one_shot, right_bound_one_shot = bootstrap(accuracies)

metric: accuracy, mean: 0.98, std: 0.03, 95% conf interval: [0.97 ,0.98]


In [20]:
print('mean:{:0.3f}, std:{:0.3f}, conf.invterval:[{:0.3f},{:0.3f}]'.format(mean_one_shot, std_one_shot, 
                                                                    left_bound_one_shot, right_bound_one_shot))

mean:0.978, std:0.026, conf.invterval:[0.970,0.985]


Train fewshot model, 5-shot, without cosine

In [21]:
episode_generator = fewshot_dataset.get_fewshot_generator(n_way=n_way, 
                                                          k_shot=5, 
                                                          query_size=query_size,
                                                          support_generator_args=support_generator_args,
                                                          query_generator_args=query_generator_args)

In [22]:
fewshot_optimizer = tf.keras.optimizers.Adam(lr=1e-3) 
fewshot_model = BaselineFewShotModel(backbone, n_way, with_cosine=False)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
                                   period=10)

Average acc: 97.62%: 100%|██████████| 50/50 [06:59<00:00,  9.52s/it]


In [23]:
mean_one_shot, std_one_shot, left_bound_one_shot, right_bound_one_shot = bootstrap(accuracies)

metric: accuracy, mean: 0.98, std: 0.03, 95% conf interval: [0.97 ,0.98]


In [24]:
print('mean:{:0.3f}, std:{:0.3f}, conf.invterval:[{:0.3f},{:0.3f}]'.format(mean_one_shot, std_one_shot, 
                                                                    left_bound_one_shot, right_bound_one_shot))

mean:0.976, std:0.027, conf.invterval:[0.969,0.983]


### Few Shot
Train fewshot model, 1-shot

In [25]:
episode_generator = fewshot_dataset.get_fewshot_generator(n_way=n_way, 
                                                          k_shot=1, 
                                                          query_size=query_size,
                                                          support_generator_args=support_generator_args,
                                                          query_generator_args=query_generator_args)

In [32]:
fewshot_model = BaselineFewShotModel(backbone, n_way, with_cosine=True)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
                                   period=10)

Average acc: 93.58%: 100%|██████████| 50/50 [14:46<00:00, 19.69s/it]


In [33]:
mean_one_shot, std_one_shot, left_bound_one_shot, right_bound_one_shot = bootstrap(accuracies)

metric: accuracy, mean: 0.94, std: 0.07, 95% conf interval: [0.92 ,0.95]


In [34]:
print('mean:{:0.3f}, std:{:0.3f}, conf.invterval:[{:0.3f},{:0.3f}]'.format(mean_one_shot, std_one_shot, 
                                                                    left_bound_one_shot, right_bound_one_shot))

mean:0.936, std:0.071, conf.invterval:[0.915,0.954]


Train fewshot model, 1-shot, without cosine

In [28]:
episode_generator = fewshot_dataset.get_fewshot_generator(n_way=n_way, 
                                                          k_shot=1, 
                                                          query_size=query_size,
                                                          support_generator_args=support_generator_args,
                                                          query_generator_args=query_generator_args)

In [29]:
fewshot_optimizer = tf.keras.optimizers.Adam(lr=1e-3) 
fewshot_model = BaselineFewShotModel(backbone, n_way, with_cosine=False)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
                                   period=10)

Average acc: 93.08%: 100%|██████████| 50/50 [11:24<00:00, 15.40s/it]


In [30]:
mean_one_shot, std_one_shot, left_bound_one_shot, right_bound_one_shot = bootstrap(accuracies)

metric: accuracy, mean: 0.93, std: 0.07, 95% conf interval: [0.91 ,0.95]


In [31]:
print('mean:{:0.3f}, std:{:0.3f}, conf.invterval:[{:0.3f},{:0.3f}]'.format(mean_one_shot, std_one_shot, 
                                                                    left_bound_one_shot, right_bound_one_shot))

mean:0.931, std:0.072, conf.invterval:[0.911,0.950]
